# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import random

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
admissions = pd.read_csv('Admission_Predict.csv')
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [3]:
admissions.columns = admissions.columns.str.rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [4]:
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
admissions.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
admissions = admissions.set_index('Serial No.')

In [7]:
admissions

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [8]:
len(set(zip(admissions['GRE Score'],admissions['CGPA'])))

385

In [9]:
# It's also 385.

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [10]:
admissions.set_index(['GRE Score','CGPA'], inplace = True)

In [11]:
admissions

,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,
337,9.65,118,4,4.5,4.5,1,0.92
316,8.00,104,3,3.0,3.5,1,0.72
322,8.67,110,3,3.5,2.5,1,0.80
314,8.21,103,2,2.0,3.0,0,0.65
330,9.34,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...
324,9.04,110,3,3.5,3.5,1,0.82
325,9.11,107,3,3.0,3.5,1,0.84
330,9.45,116,4,5.0,4.5,1,0.91


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [12]:
admissions = pd.read_csv('Admission_Predict.csv')

count = admissions[(admissions['CGPA']>9)&(admissions['Research']>0)]
print('The number of rows that meet this condition is: ' + str(len(count)))

The number of rows that meet this condition is: 101


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [13]:
count2 = admissions[(admissions['CGPA']>9)&(admissions['SOP']<3.5)]
count2

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
28,29,338,118,4,3.0,4.5,9.40,1,0.91
62,63,327,114,3,3.0,3.0,9.02,0,0.61
140,141,326,114,3,3.0,3.0,9.11,1,0.83
217,218,324,111,4,3.0,3.0,9.01,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84


In [14]:
count2['Chance of Admit '].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [15]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    col = pd.Series((col-np.mean(col))/np.std(col))
    return col

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [16]:
admissions['CGPA_std'] = standardize(admissions['CGPA'])
admissions['GRE_std'] =standardize(admissions['GRE Score'])
admissions['LOR_std'] = standardize(admissions['LOR '])

admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019


We will generate the decision choice at random using the code below. Please run the cell.

In [17]:
# Libraries
from random import choices

In [18]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [19]:
admissions['deciding_column'] = admissions.lookup(admissions.index, decision_choice)
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.716170
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,-0.409947


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [20]:
admissions['decision'] = np.where(admissions['deciding_column']>0.8, '1', '0')
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.716170,0
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,-0.409947,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [21]:
len(admissions[(admissions['decision']=='1')])

96

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [22]:
admissions.columns = admissions.columns.str.replace('[^a-zA-Z]', ' ').str.replace(' ', '_').str.lower()
admissions

,serial_no_,gre_score,toefl_score,university_rating,sop,lor_,cgpa,research,chance_of_admit_,cgpa_std,gre_std,lor_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.716170,0
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,-0.409947,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [23]:
admissions.loc[admissions['university_rating'] >= 4, 'gre_score'] = admissions['gre_score']+10

In [27]:
admissions['adjusted_gre'] = pd.cut(admissions['gre_score'], bins=4, labels=['1','2','3','4'])
admissions

,serial_no_,gre_score,toefl_score,university_rating,sop,lor_,cgpa,research,chance_of_admit_,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,1,347,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.755663,1,4
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0,2
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0,3
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0,2
4,5,340,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.149292,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0,3
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.716170,0,3
382,383,340,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.193197,1,4
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,-0.409947,0,2
